In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

In [2]:
# 환경 변수 불러오기
load_dotenv()

# .env 파일에서 정보 읽기
USER_ID = os.getenv('API_USER_ID')
API_KEY = os.getenv('API_KEY')
API_FORMAT_NUMBER = os.getenv('API_FORMAT_NUMBER')

In [ ]:
# print(USER_ID)
# print(API_KEY)
# print(API_FORMAT_NUMBER)

In [3]:
def fetch_data(year):
    """API에서 특정 연도의 데이터를 가져오는 함수"""
    base_url = "https://www.recycling-info.or.kr/sds/JsonApi.do"
    params = {
        'PID': API_FORMAT_NUMBER,
        'YEAR': year,
        'USRID': USER_ID,
        'KEY': API_KEY
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for year {year}: {response.status_code}")
        return None

In [4]:
def create_dataframe(data, year):
    """주어진 데이터를 데이터프레임으로 변환하고 컬럼명을 한글로 변경하는 함수, 빈 데이터 처리 포함"""
    if data and 'data' in data and data['data']:
        data_items = data['data']
        headers = data['dataHeader'][0]

        # 연도에 따른 매핑 설정
        if year >= 2019:
            # 2019년 이상 데이터 매핑
            column_mapping = {
                'CITY_JIDT_CD_NM': '시도',
                'CTS_JIDT_CD_NM': '시군구',
                'WT_TYPE_GB_NM': '폐기물 대분류',
                'WSTE_M_CODE_NM': '폐기물 중분류',
                'WSTE_CODE_NM': '폐기물 소분류',
                'WSTE_QTY': '전체발생량',
                'TOT_RECY_QTY': '총계_재활용',
                'TOT_INCI_QTY': '총계_소각',
                'TOT_FILL_QTY': '총계_매립',
                'TOT_ETC_QTY': '총계_기타'
            }
        else:
            # 2018년 이전 데이터 매핑
            column_mapping = {
                'CITY_JIDT_NM': '시도',
                'CTS_JIDT_NM': '시군구',
                'TOT_SUM_QTY': '전체발생량',
                'TOT_RECY_QTY': '총계_재활용',
                'TOT_INCI_QTY': '총계_소각',
                'TOT_FILL_QTY': '총계_매립'
            }

        # 데이터프레임 생성
        df = pd.DataFrame(data_items)
        
        # 컬럼명 변경
        df.rename(columns=column_mapping, inplace=True)

        # 2018년 이전 데이터에 대한 추가 컬럼 설정
        if year < 2019:
            df['폐기물 대분류'] = ''
            df['폐기물 중분류'] = ''
            df['폐기물 소분류'] = ''
            df['총계_기타'] = 0  # 총계_기타에 0 채우기

        # 최종적으로 선택할 컬럼
        final_columns = [
            '시도', '시군구', '폐기물 대분류', '폐기물 중분류', '폐기물 소분류',
            '전체발생량', '총계_재활용', '총계_소각', '총계_매립', '총계_기타'
        ]
        df = df[final_columns]  # 필요한 컬럼만 선택
    else:
        # 데이터가 없는 경우 표준 컬럼으로 빈 데이터프레임 생성
        df = pd.DataFrame(columns=final_columns)
    
    df.insert(0, '발생년도', year)  # '발생년도' 컬럼을 첫 번째 위치에 추가
    return df

In [21]:
def main():
    # 데이터를 가져올 연도 범위 설정
    years = range(2014, 2024)  # 예: 2020년부터 2023년까지
    
    # 연도별 데이터 프레임을 저장할 리스트
    data_frames = []
    
    for year in years:
        data = fetch_data(year)
        # data에 헤더만 존재하고 data['data']에 구체적인 데이터가 없을 경우 건너뛰기
        # data['data'][0]에서 {'PRIMARYVAL': 'NODATA'} 일때 해당 데이터는 없는 것으로 판단
        # print(data['data'][0])
        # if data and 'data' in data and data['data'] and data['data'][0]['PRIMARYVAL'] == 'NODATA':
        #     print(f"No data found for year {year}")
        #     continue
       
        df = create_dataframe(data, year)
        df.reset_index(drop=True, inplace=True)  # 인덱스 리셋
        data_frames.append(df)
    
    # 모든 데이터 프레임을 하나로 합치기
    final_df = pd.concat(data_frames, ignore_index=True)

    # Excel 파일로 저장
    final_df.to_excel('data/combined_data.xlsx', index=False, engine='openpyxl')    
    # 결과를 CSV 파일로 저장
    # final_df.to_csv('data/combined_data.csv', index=False, encoding='cp949', sep=';', quotechar='"')
    print("Data has been successfully saved to 'data/combined_data.csv'")

In [22]:
if __name__ == "__main__":
    main()

Data has been successfully saved to 'data/combined_data.csv'
